In [15]:
import os
import logging
import pandas as pd
from bs4 import BeautifulSoup
import httpx

In [16]:
# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%d/%m/%Y %H:%M:%S'
)
logger = logging.getLogger(__name__)

In [19]:
# URL do site
url = "https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php"

def get_html(url: str, verify_ssl: bool = True) -> str:
    """
    Retorna o conteúdo HTML da página.

    Parâmetros:
        url (str): URL do site.
        verify_ssl (bool): Se False, ignora a verificação SSL.

    Retorna:
        str: Conteúdo HTML da página.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    logger.debug(f"Fazendo requisição para {url}")
    
    try:
        response = httpx.get(url, headers=headers, timeout=10, verify=verify_ssl)
        response.raise_for_status()
        logger.debug("Requisição bem-sucedida")
        return response.text
    except httpx.HTTPStatusError as e:
        logger.error(f"Erro HTTP ao fazer requisição: {e.response.status_code} - {e.request.url}")
        raise
    except httpx.RequestError as e:
        logger.error(f"Erro na requisição: {e}")
        raise

def extract_table_data(html: str) -> list[list[str]]:
    """
    Extrai os dados da tabela HTML e retorna uma lista de listas.

    Parâmetros:
        html (str): Conteúdo HTML da página.

    Retorna:
        list[list[str]]: Cada sublista contém os dados de uma linha da tabela.
    """
    logger.info("Extraindo os dados da tabela HTML")
    
    soup = BeautifulSoup(html, 'html.parser')
    table_rows = soup.find_all('tr')

    data = []
    for row in table_rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols if col.text.strip()]
        if cols and cols[0].isdigit():  # Ignora linhas sem dados relevantes
            data.append(cols)
        elif len(cols) == 1 and cols[0] == "Year":
            logger.debug("Ignorando linha repetida de década")
    
    if not data:
        logger.warning("Nenhuma linha de dados foi extraída.")
    else:
        logger.info(f"{len(data)} linhas extraídas com sucesso.")
    
    return data

def create_dataframe(data: list[list[str]]) -> pd.DataFrame:
    """
    Cria um DataFrame a partir dos dados extraídos.

    Parâmetros:
        data (list[list[str]]): Lista de listas com os dados da tabela.

    Retorna:
        pd.DataFrame: DataFrame com os dados.
    """
    logger.debug("Criando DataFrame a partir dos dados extraídos")
    columns = ["Year", "DJF", "JFM", "FMA", "MAM", "AMJ", "MJJ", "JJA", "JAS", "ASO", "SON", "OND", "NDJ"]
    
    try:
        df = pd.DataFrame(data, columns=columns)
        df[columns[1:]] = df[columns[1:]].apply(pd.to_numeric, errors='coerce')
        df.dropna(inplace=True)
        logger.debug("DataFrame criado com sucesso")
        return df
    except Exception as e:
        logger.error(f"Erro ao criar DataFrame: {e}")
        raise

def calcular_media_mes(df: pd.DataFrame, mes_colunas: list[str]) -> pd.Series:
    """Calcula a média dos valores dos meses passados por ano."""
    logger.debug(f"Calculando a média para os meses: {mes_colunas}")
    return df[mes_colunas].mean(axis=1)

def classificar_fenomeno(media: float) -> tuple[str, str]:
    """Classifica o fenômeno El Niño ou La Niña."""
    if media > 0.5:
        if media >= 1.5:
            return 'El Niño', 'Forte'
        elif media >= 1.0:
            return 'El Niño', 'Moderado'
        return 'El Niño', 'Fraco'
    elif media < -0.5:
        if media <= -1.5:
            return 'La Niña', 'Forte'
        elif media <= -1.0:
            return 'La Niña', 'Moderado'
        return 'La Niña', 'Fraco'
    return 'Neutro', 'Neutro'

def obter_trimestres_para_mes(mes: str) -> list[str]:
    """
    Mapeia o mês para os três trimestres correspondentes.
    """
    mapeamento = {
        'Janeiro': ['NDJ', 'DJF', 'JFM'],
        'Fevereiro': ['DJF', 'JFM', 'FMA'],
        'Março': ['JFM', 'FMA', 'MAM'],
        'Abril': ['FMA', 'MAM', 'AMJ'],
        'Maio': ['MAM', 'AMJ', 'MJJ'],
        'Junho': ['AMJ', 'MJJ', 'JJA'],
        'Julho': ['MJJ', 'JJA', 'JAS'],
        'Agosto': ['JJA', 'JAS', 'ASO'],
        'Setembro': ['JAS', 'ASO', 'SON'],
        'Outubro': ['ASO', 'SON', 'OND'],
        'Novembro': ['SON', 'OND', 'NDJ'],
        'Dezembro': ['OND', 'NDJ', 'DJF'],
    }
    return mapeamento.get(mes, [])

def analisar_ano(df: pd.DataFrame) -> pd.DataFrame:
    """
    Analisa os fenômenos El Niño e La Niña para todos os meses de cada ano.
    """
    logger.info("Analisando os fenômenos El Niño e La Niña por mês e ano")
    resultados = []
    
    for _, row in df.iterrows():
        ano = row['Year']
        
        for mes in ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 
                    'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']:
            
            trimestres = obter_trimestres_para_mes(mes)
            media_mes = calcular_media_mes(df.iloc[[_]], trimestres).iloc[0]
            fenomeno, intensidade = classificar_fenomeno(media_mes)

            logger.info(f"Ano {ano}, Mês {mes}: Fenômeno {fenomeno}, Intensidade {intensidade}")
            resultados.append([ano, mes, fenomeno, intensidade])
    
    df_resultados = pd.DataFrame(resultados, columns=['Ano', 'Mês', 'Fenômeno', 'Intensidade'])
    logger.info("Análise concluída")
    return df_resultados

def save_to_excel(df: pd.DataFrame, folder_path: str, file_name: str):
    """Salva o DataFrame em um arquivo Excel em uma pasta específica."""
    os.makedirs(folder_path, exist_ok=True)  # Cria a pasta se não existir
    file_path = os.path.join(folder_path, file_name)  # Cria o caminho completo
    
    try:
        df.to_excel(file_path, index=False)
        logger.info(f"Arquivo Excel salvo com sucesso em {file_path}")
    except Exception as e:
        logger.error(f"Erro ao salvar o arquivo Excel: {e}")
        raise

def main(folder_path: str = r"C:\Users\anaph\OneDrive\Área de Trabalho\DadosMeteorologicos", 
         excel_file: str = "resultados_oni.xlsx"):
    """Script principal para extração e análise dos dados do ONI."""
    try:
        logger.info("Iniciando o processo de extração e análise")
        
        # Etapa 1: Requisição e extração dos dados
        html = get_html(url, verify_ssl=True)
        table_data = extract_table_data(html)

        if not table_data:
            logger.warning("Nenhum dado encontrado para processar.")
            return

        # Etapa 2: Criar DataFrame
        df = create_dataframe(table_data)

        # Etapa 3: Analisar e classificar os fenômenos
        df_resultados = analisar_ano(df)

        # Etapa 4: Salvar em Excel
        save_to_excel(df_resultados, folder_path, excel_file)
        logger.info("Processo concluído com sucesso")

    except Exception as e:
        logger.error(f"Erro durante a execução: {e}")

if __name__ == "__main__":
    main()

08/10/2024 22:06:17 - INFO - Iniciando o processo de extração e análise
08/10/2024 22:06:19 - INFO - HTTP Request: GET https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php "HTTP/1.1 200 OK"
08/10/2024 22:06:19 - INFO - Extraindo os dados da tabela HTML
08/10/2024 22:06:19 - INFO - 75 linhas extraídas com sucesso.
08/10/2024 22:06:19 - INFO - Analisando os fenômenos El Niño e La Niña por mês e ano
08/10/2024 22:06:19 - INFO - Ano 1950, Mês Janeiro: Fenômeno La Niña, Intensidade Moderado
08/10/2024 22:06:19 - INFO - Ano 1950, Mês Fevereiro: Fenômeno La Niña, Intensidade Moderado
08/10/2024 22:06:19 - INFO - Ano 1950, Mês Março: Fenômeno La Niña, Intensidade Moderado
08/10/2024 22:06:19 - INFO - Ano 1950, Mês Abril: Fenômeno La Niña, Intensidade Moderado
08/10/2024 22:06:19 - INFO - Ano 1950, Mês Maio: Fenômeno La Niña, Intensidade Moderado
08/10/2024 22:06:19 - INFO - Ano 1950, Mês Junho: Fenômeno La Niña, Intensidade Fraco
08/10/2024 22:06:19 - INFO - Ano 1